In [278]:
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
tokens = text.encode("utf-8")
tokens = list(map(int, tokens))

In [279]:
def pair_count(data):
    count = {}
    for pair in zip(data, data[1:]):
        count[pair] = count.get(pair, 0) + 1
    return count

In [280]:
def merge(ids, pair, idx):
    """Replaces the `pair` with the `idx` whenever it appears in the `ids`"""
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and (ids[i], ids[i + 1]) == pair:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

In [281]:
vocab_size = 276
num_merges = vocab_size - 256
ids = list(tokens)

merges = {}
for i in range(num_merges):
    stats = pair_count(ids)
    pair = max(stats, key=stats.get)
    idx = 256 + i
    ids = merge(ids, pair, idx)
    print(f"merging pair {pair} into a new token {idx}")
    merges[pair] = idx

merging pair (101, 32) into a new token 256
merging pair (240, 159) into a new token 257
merging pair (226, 128) into a new token 258
merging pair (105, 110) into a new token 259
merging pair (115, 32) into a new token 260
merging pair (97, 110) into a new token 261
merging pair (116, 104) into a new token 262
merging pair (257, 133) into a new token 263
merging pair (257, 135) into a new token 264
merging pair (97, 114) into a new token 265
merging pair (239, 189) into a new token 266
merging pair (258, 140) into a new token 267
merging pair (267, 264) into a new token 268
merging pair (101, 114) into a new token 269
merging pair (111, 114) into a new token 270
merging pair (116, 32) into a new token 271
merging pair (259, 103) into a new token 272
merging pair (115, 116) into a new token 273
merging pair (261, 100) into a new token 274
merging pair (32, 262) into a new token 275


In [282]:
print(len(tokens))
print(len(ids))
print(f"{len(tokens) / len(ids):.2f}x compression")

616
451
1.37x compression


In [287]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]


def decode(ids):
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace")
    return text

In [288]:
def encode(text):
    tokens = list(text.encode("utf-8"))
    while len(tokens) >= 2:
        stats = pair_count(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))
        if pair not in merges:
            break
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

In [ ]:
print(decode(encode(text)))

Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.


In [1]:
import regex as re

gpt2pat = re.compile(
    r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
)

print(re.findall(gpt2pat, "Hello've world123 how's are you!!!?"))

['Hello', "'ve", ' world', '123', ' how', "'s", ' are', ' you', '!!!?']


In [4]:
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")
enc.encode("whats good with you")

[1336, 1900, 1695, 449, 499]